Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
import pandas as pd
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical

In [2]:
data = pd.read_excel('отзывы за лето.xls')
data

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [3]:
max_words = 20000
max_len = 150
num_classes = 5

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [4]:
stopwordslist = stopwords.words("russian")
morpher = MorphAnalyzer()
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [5]:
data['Content_norm'] = data['Content'].apply(normalize_text)

In [6]:
data.sample(5)

,Rating,Content,Date,Content_norm
10354,5,Отлично,2017-07-30,отлично
15019,5,Всё классно,2017-07-22,вс классно
17189,5,Очень удобное приложение,2017-07-13,очень удобный приложение
18077,4,Все ОК,2017-07-07,около
20060,5,"Всё замечательно, спасибо!",2017-06-15,вс замечательный спасибо


In [7]:
train_corpus = " ".join(data['Content_norm'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [8]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'очень',
 'удобно',
 'работать',
 'удобный',
 'вс',
 'спасибо',
 'отлично',
 'нравиться',
 'телефон']

In [9]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [10]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [11]:
data_train = np.asarray([text_to_sequence(text, max_len) for text in data['Content_norm']], dtype=np.int32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.33, random_state=42)

In [13]:
X_train.shape

(13841, 150)

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)
le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

### Эмбединги по умолчанию

In [15]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping

In [16]:
epochs = 5
batch_size = 32

In [17]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 128)          2560000   
_________________________________________________________________
conv1d (Conv1D)              (None, 148, 128)          49280     
_________________________________________________________________
activation (Activation)      (None, 148, 128)          0         
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0

In [19]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.1)

Epoch 1/5
390/390 [==============================] - 25s 21ms/step - loss: 0.8921 - accuracy: 0.7290 - val_loss: 0.6807 - val_accuracy: 0.7588
Epoch 2/5
390/390 [==============================] - 7s 18ms/step - loss: 0.5770 - accuracy: 0.7904 - val_loss: 0.6734 - val_accuracy: 0.7625
Epoch 3/5
390/390 [==============================] - 7s 18ms/step - loss: 0.5277 - accuracy: 0.8068 - val_loss: 0.6751 - val_accuracy: 0.7675
Epoch 4/5
390/390 [==============================] - 7s 18ms/step - loss: 0.4543 - accuracy: 0.8380 - val_loss: 0.7035 - val_accuracy: 0.7690
Epoch 5/5
390/390 [==============================] - 7s 19ms/step - loss: 0.3904 - accuracy: 0.8585 - val_loss: 0.7590 - val_accuracy: 0.7653


In [21]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

214/214 [==============================] - 1s 3ms/step - loss: 0.7378 - accuracy: 0.7727


Test score: 0.737763524055481
Test accuracy: 0.7726606130599976


### Предобученнные эмбединги

In [22]:
path_to_file = "./182/model.txt"

embeddings_index = {}
with open(path_to_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 248979 word vectors.


In [23]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [24]:
model = Sequential()
model.add(Embedding(input_dim=max_words, weights=[embedding_matrix], output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          6000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 128)          115328    
_________________________________________________________________
activation_3 (Activation)    (None, 148, 128)          0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [26]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.1)

Epoch 1/5
390/390 [==============================] - 18s 44ms/step - loss: 1.5234 - accuracy: 0.6936 - val_loss: 1.3041 - val_accuracy: 0.7011
Epoch 2/5
390/390 [==============================] - 17s 43ms/step - loss: 1.2540 - accuracy: 0.6984 - val_loss: 1.1324 - val_accuracy: 0.7011
Epoch 3/5
390/390 [==============================] - 17s 43ms/step - loss: 1.0906 - accuracy: 0.7128 - val_loss: 1.0531 - val_accuracy: 0.7011
Epoch 4/5
390/390 [==============================] - 17s 44ms/step - loss: 1.0378 - accuracy: 0.7021 - val_loss: 1.0198 - val_accuracy: 0.7011
Epoch 5/5
390/390 [==============================] - 17s 43ms/step - loss: 1.0160 - accuracy: 0.6991 - val_loss: 1.0068 - val_accuracy: 0.7011


In [29]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

214/214 [==============================] - 1s 3ms/step - loss: 0.9802 - accuracy: 0.7125


Test score: 0.9802106618881226
Test accuracy: 0.7125256657600403


Точность на предобученных эмбедингах ниже. Вероятно, сказывается несоответсвие словарей.